<a href="https://colab.research.google.com/github/babaroscopy/Thesis/blob/main/multifiltercnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.mkdir('/content/normal+bags')
os.mkdir('/content/Normal+Coats')
os.mkdir('/content/NormalToCoats')
os.mkdir('/content/NormalToNormal')
os.mkdir('/content/SpeedExperiments')
os.mkdir('/content/SpeedExperiments/Normal+Fast')
os.mkdir('/content/SpeedExperiments/Normal+Slow')
os.mkdir('/content/SpeedExperiments/NormalToNormal')

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
from imageio import imread
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing import image
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten,LeakyReLU,BatchNormalization
from keras.applications import densenet
from keras.regularizers import l1
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.layers import ZeroPadding2D
from keras import optimizers
from matplotlib import pyplot
import sys
import pandas as pd
import numpy

In [ ]:
numpy.set_printoptions(threshold=sys.maxsize)
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]

images1 = "/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/TrainNormal"
files1= os.listdir(images1)
#print(files1)
X_train = [imread('/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/TrainNormal/'+f) for f in files1]   # Getting Training Images
for f in files1:
    Y_train.append(f[7:10])
for i in range(0, len(Y_train)):
    Y_train[i] = int(Y_train[i])
#print(Y_train)
images2 = '/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/TestCoats+Normal-C'
files2= os.listdir(images2)
#print(files2)
X_test = [imread("/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/TestCoats+Normal-C/"+f) for f in files2]
for f in files2:
    Y_test.append(f[7:10])

for i in range(0, len(Y_test)):
    Y_test[i] = int(Y_test[i])
#print(Y_test)

X_train=np.array(X_train)
X_test=np.array(X_test)

img_rows=240
img_cols=240
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

    input_shape = (img_rows, img_cols, 1)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_test=label_encoder.fit_transform(Y_test)
Y_train= label_encoder.fit_transform(Y_train)

print(Y_train)
print(Y_test)

from keras.utils import to_categorical
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
num_classes = 124
epochs = 50
MODEL_NAME = 'TrainedModelFile-{}-{}.model'.format("0.0001", 'TenLayers')
# Define the custom kernel weights for the first layer
first_kernel_weights = [[0,0,0], [0,1,0], [0,0,0]]

# Define the custom kernels for different parts of the image
kernel1 = np.array([[-1, 0.1, 0.5],
                    [-1, 1.5, 0.5],
                    [-1, 0.1, 0.5]])

kernel2 = np.array([[-0.5, -0.1, 0.5],
                    [-0.5, 0.3, 0.5],
                    [-0.5, -0.1, 0.5]])

kernel3 = np.array([[-0.5, -0.1, 0.5],
                    [-0.5, 0.3, 0.5],
                    [-0.5, -0.1, 0.5]])

kernel4 = np.array([[-0.5, -0.1, 0.5],
                    [-0.5, 0.3, 0.5],
                    [-0.5, -0.1, 0.5]])

kernel5 = np.array([[-0.6, -0.4, -0.6],
                    [-0.4, 5.2, -0.4],
                    [-0.6, -0.4, -0.6]])

# Define the CNN model
model = Sequential()

model.add(Conv2D(1, (3, 3), input_shape=(image_height, image_width, 1), padding='same', kernel_initializer=tf.constant_initializer(first_kernel_weights)))

'''
# Split the input shape into three parts
split_height = image_height // 3


# Convolution with kernel1 for region 1
region1 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel1))(model.output[:, :split_height, :, :])

# Convolution with kernel2 for region 2
region2 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel2))(model.output[:, split_height:2*split_height, :, :])

# Convolution with kernel3 for region 3
region3 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel3))(model.output[:, 2*split_height:, :, :])

# Merge the regions back into a single tensor
merged_regions = tf.concat([region1, region2, region3], axis=1)
'''

'''
#4 parts and filters
split_height = image_height // 4

# Convolution with kernel1 for region 1
region1 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel1))(model.output[:, :split_height, :, :])

# Convolution with kernel2 for region 2
region2 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel2))(model.output[:, split_height:2*split_height, :, :])

# Convolution with kernel3 for region 3
region3 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel3))(model.output[:, 2*split_height:3*split_height, :, :])

# Convolution with kernel4 for region 4
region4 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel4))(model.output[:, 3*split_height:, :, :])

# Merge the regions back into a single tensor
merged_regions = tf.concat([region1, region2, region3, region4], axis=1)
'''

'''
#6 parts and kernels
split_height = image_height // 6

# Convolution with kernel1 for region 1
region1 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel1))(model.output[:, :split_height, :, :])

# Convolution with kernel2 for region 2
region2 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel2))(model.output[:, split_height:2*split_height, :, :])

# Convolution with kernel3 for region 3
region3 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel3))(model.output[:, 2*split_height:3*split_height, :, :])

# Convolution with kernel4 for region 4
region4 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel4))(model.output[:, 3*split_height:4*split_height, :, :])

# Convolution with kernel5 for region 5
region5 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel5))(model.output[:, 4*split_height:5*split_height, :, :])

# Convolution with kernel6 for region 6
region6 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel6))(model.output[:, 5*split_height:, :, :])

# Merge the regions back into a single tensor
merged_regions = tf.concat([region1, region2, region3, region4, region5, region6], axis=1)

'''

'''
#7 parts and kernels
split_height = image_height // 7

# Convolution with kernel1 for region 1
region1 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel1))(model.output[:, :split_height, :, :])

# Convolution with kernel2 for region 2
region2 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel2))(model.output[:, split_height:2*split_height, :, :])

# Convolution with kernel3 for region 3
region3 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel3))(model.output[:, 2*split_height:3*split_height, :, :])

# Convolution with kernel4 for region 4
region4 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel4))(model.output[:, 3*split_height:4*split_height, :, :])

# Convolution with kernel5 for region 5
region5 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel5))(model.output[:, 4*split_height:5*split_height, :, :])

# Convolution with kernel6 for region 6
region6 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel6))(model.output[:, 5*split_height:6*split_height, :, :])

# Convolution with kernel7 for region 7
region7 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel7))(model.output[:, 6*split_height:, :, :])

# Merge the regions back into a single tensor
merged_regions = tf.concat([region1, region2, region3, region4, region5, region6, region7], axis=1)


'''
split_height = image_height // 5

# Convolution with kernel1 for region 1
region1 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel1))(model.output[:, :split_height, :, :])

# Convolution with kernel2 for region 2
region2 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel2))(model.output[:, split_height:2*split_height, :, :])

# Convolution with kernel3 for region 3
region3 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel3))(model.output[:, 2*split_height:3*split_height, :, :])

# Convolution with kernel4 for region 4
region4 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel4))(model.output[:, 3*split_height:4*split_height, :, :])

# Convolution with kernel5 for region 5
region5 = Conv2D(1, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(kernel5))(model.output[:, 4*split_height:, :, :])

# Merge the regions back into a single tensor
merged_regions = tf.concat([region1, region2, region3, region4, region5], axis=1)

model.add(LeakyReLU(alpha=0.05))
#model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(16,3, 3))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, 3, 3))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, 3, 3))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(124, 3, 3,padding='same'))
model.add(LeakyReLU(alpha=0.05))
model.add(ZeroPadding2D(padding=((0,1),(0,1))))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1024))
model.add(LeakyReLU(alpha=0.05))
batch_size = 4

model.add(Dense(124, activation='softmax'))


print(model.summary())
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

import time
start_time=time.time()
history=model.fit(X_train, Y_train,batch_size=4, epochs=epochs,validation_data=(X_test, Y_test))
#history=model.fit(X_train, Y_train,batch_size=4, epochs=epochs)
print('Training took {} seconds'.format(time.time()-start_time))
#model.save(MODEL_NAME)
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()
plt.savefig('/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/Curves.png')

from sklearn.metrics import accuracy_score

test_predictions = model.predict(X_test)


accuracy = accuracy_score(np.argmax(Y_test,axis=1), np.argmax(test_predictions,axis=1))*100
print("Accuracy: " + str(accuracy))
from sklearn.metrics import precision_score, recall_score
precision_score= precision_score(np.argmax(Y_test,axis=1), np.argmax(test_predictions,axis=1), average=None)

recall_score= recall_score(np.argmax(Y_test,axis=1),np.argmax(test_predictions,axis=1),average=None)
from sklearn.metrics import f1_score
F1=f1_score(np.argmax(Y_test,axis=1),np.argmax(test_predictions,axis=1), average=None)
from sklearn.metrics import confusion_matrix
results = confusion_matrix(np.argmax(Y_test,axis=1),np.argmax(test_predictions,axis=1))
print(results)
from sklearn.metrics import classification_report
list3=[]
for i in range(1,125):
    list3.append("Person"+str(i))
#print(list3)
print(classification_report(np.argmax(Y_test,axis=1), np.argmax(test_predictions,axis=1)))

list3=[]
for i in range(1,125):
    list3.append("Person"+str(i))
#print(list3)


labels=pd.DataFrame()
labels["ClassLabel"]=list3
labels["Precision"]=precision_score
labels["Recallvalue"]=recall_score
labels["F1Score"]=F1
labels.to_csv('/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/File.csv')

'''
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy Results')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train'], loc='upper left')
plt.savefig('/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/AccuracyCurve.png')
plt.show()
plt.close()
'''
'''
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('Model Loss Results')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train'], loc='upper right')
plt.savefig('/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/LossCurve.png')
plt.show()
plt.close()
'''
fig, ax = plot_confusion_matrix(conf_mat=results,colorbar=False,show_absolute=True,show_normed=False,figsize=(40,40))
plt.savefig('/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/ConfusionMatrix.png')
plt.show()
plt.close()
arr = np.array(results)

df = pd.DataFrame(arr)

df.columns = list3
df["Rows/Col"]=list3
df.to_csv('/content/drive/MyDrive/DatasetGEIs/DatasetGEIs/Normal+Coats/File2.csv')
